In [51]:

# 고객센터 - 
import os
import json
from openai import OpenAI
client = OpenAI()

# 가상의 고객 데이터
customer_reviews = [
    {'id' : 1,
     'product' : '무선 청소기',
     'date' : '2025-01-10',
     'content' : '너무 시끄럽고 충전이 잘 안되요 그냥 환불 하고 싶습니다. 박스는 버렸어요'
     },
    {'id' : 2,
     'product' : '여름용 셔츠',
     'date' : '2025-02-10',
     'content' : '사이즈가 광고랑 달라요 배송도 느리고 교환 가능한가요? 구매한지는 한달이 조금 넘었습니다.'
     },
    {'id' : 3,
     'product' : '블루투스 키보드',
     'date' : '2025-03-10',
     'content' : '블루투스가 잘 안잡혀요. 아니 인식이 안되는거 같아요.. 교환 또는 환불해주세요. 박스는 있어요'
     },
]
# 분류 - 판단 - 대응
# 회사의 환불/보상 규정( Knowledge Base for RAG)
policy_doc = '''
[다판다 교환/환불 및 보상 규정]
1. 단순 변심 환불 : 구매후 7일 이내 가능, 단 제품 박스가 온전해야 함
2. 제품 불량 환불 : 구매후 30일 이내 가능, 박스유무 상관 없음
3. 불만족 보상 : 구매 확정후에는 환불 불가, 소정의 적립금(2,000원) 지급 가능
4. AS 규정 : 구매 후 1년 이내 무상 AS 가능
'''

def ask_gpt_json(prompt):
    response = client.chat.completions.create(model = 'gpt-5-nano',
                                   messages=[{'role':'user','content':prompt}],
                                   response_format={'type':'json_object'}
                                   )
    try:
        return response.choices[0].message.content
    except:
        return None

In [52]:
# 단순히 요약이 아닌 카테고리, 감정점수, 핵심요구사항을 뽑아야함
def analyze_review(text):
    prompt = f'''
    너는 쇼핑몰 리뷰 전문가야, 아래 리뷰를 분석해서 다음 json 형식으로 출력
    
    [출력형식]
    {{
        'sentiment' : '긍정/부정/중립 중 하나',
        'category' : '배송/품질/서비스/기타 중 하나',
        'request_type' : '환불/교환/보상/AS/단순후기 중 하나',
        'key_issue' : "핵심 불만 사항(10자 이내)"
    }}

    [리뷰]
    {text}
    '''
    return json.loads(ask_gpt_json(prompt))  # 문자열 형태의 dict를 실제 객체로 변환

analzed_data_list = []
for item in customer_reviews:
    analysis = analyze_review( item['content'] )
    # 원본데이터에 분석 결과 합치기
    merged_data = {**item, **analysis}
    analzed_data_list.append(merged_data)

analzed_data_list

[{'id': 1,
  'product': '무선 청소기',
  'date': '2025-01-10',
  'content': '너무 시끄럽고 충전이 잘 안되요 그냥 환불 하고 싶습니다. 박스는 버렸어요',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '환불',
  'key_issue': '소음과충전불량'},
 {'id': 2,
  'product': '여름용 셔츠',
  'date': '2025-02-10',
  'content': '사이즈가 광고랑 달라요 배송도 느리고 교환 가능한가요? 구매한지는 한달이 조금 넘었습니다.',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '사이즈 불일치'},
 {'id': 3,
  'product': '블루투스 키보드',
  'date': '2025-03-10',
  'content': '블루투스가 잘 안잡혀요. 아니 인식이 안되는거 같아요.. 교환 또는 환불해주세요. 박스는 있어요',
  'sentiment': '부정',
  'category': '품질',
  'request_type': '교환',
  'key_issue': '블루투스 인식 불가'}]

In [53]:
# ai가 멋대로 판단하지 않고 근거를 기반으로 생각하게 함
def make_decision(customer_data, policy_doc):
    prompt = f'''
        너는 베테랑 CS 매니저야
        아래 [고객 데이터]와 [회사규정]을 비교해서 요구사항을 들어줄 수 있는지 판단해

        [회사규정]
        {policy_doc}

        [고객데이터]
        - 구매일 : {customer_data['date']}
        - 오늘날짜 : 2025-11-24(기준)
        - 요청유형 : {customer_data['request_type']}
        - 내용 : {customer_data['content']}

        [지시사항]
        1. 규정의 몇 번 조항에 해당되는지 먼저 생각할 것(Cot)
        2. 날자를 계산해서 기간 내인지 확인할 것
        3. 최종 결정을 json으로 출력할 것

        [출력형식]
        {{
            'reasoning':'판단 근거(단계별 서술)',
            'decision':'승인/거절',
            'action_guide' : '상담원이 해야 할 행동 가이드'
        }}
    '''
    return json.loads(ask_gpt_json(prompt))

decision_results = []
for data in analzed_data_list:
    decision = make_decision(data,policy_doc)
    # 결과 저장
    final_result = {**data, 'decision_info':decision}
    decision_results.append(final_result)

    print(f'{data['id']} 판단결과')
    print(f'사유 : {decision['reasoning']}')
    print(f'결정 : {decision['decision']}')
    print(f'행동 가이드 : {decision['action_guide']}')
    print('-'*100)

1 판단결과
사유 : 1) 적용 조항 판단: - 단순 변심 환불(조항 1)은 박스가 온전해야 한다는 조건이 있어 현재 박스가 없으므로 적합하지 않음. - 제품 불량 환불(조항 2)은 박스 유무와 무관하나 30일 이내에 가능하므로 이미 기간 초과로 불가. - 불만족 보상(조항 3)은 환불이 불가하더라도 소정의 적립금 지급 가능 여부를 검토하나 이 경우도 구체적 지급 조건이 운영 가이드에 따름. - AS 규정(조항 4)은 구매 후 1년 이내 무상 AS 가능으로 현재 상황에서 가장 타당한 해결 경로임. 따라서 환불 요구는 규정상 거절하되, AS를 통해 문제를 해결하는 방향이 적절함.
결정 : 거절/환불 불가, 대신 AS 가능(무상) 안내
행동 가이드 : 상담원이 따라야 할 절차: 1) 고객의 불편 사항에 공감 표명: 소음과 충전 문제에 대해 이해한다고 전하기. 2) 규정 요약 및 근거 설명: 현재 환불은 기간 초과로 불가, 단 AS는 1년 이내 무상 적용 가능함을 명확히 안내. 3) 가능 경로 제시: 1년 이내 무상 AS 접수 진행 안내. 필요한 정보 수집: 주문번호, 제품 모델, 시리얼 번호, 구매자 정보, 문제 발생 상황(소음 정도, 충전 문제 재현 여부). 4) AS 절차 안내: 접수 방법(온라인/전화), 서비스 센터 위치, 수리 예상 기간, 배송/수리 방식(수리비 여부 포함). 5) 박스 유무는 AS에는 영향이 없으므로 박스 유무와 관계 없이 AS 진행 가능하다고 안내. 6) 고객 선택에 따른 후속 조치: AS 접수 진행 후 상태 업데이트 제공. 7) 만약 고객이 특별 예외를 요청할 경우에는 상위 정책 여부 확인 및 필요 시 supervisor 승인을 요청하는 절차 안내. 8) 추가 대안 검토: 불만족 보상으로 소정의 적립금 지급 가능 여부를 확인해 필요 시 안내(정책상 가능 여부를 내부 가이드에 따름).
--------------------------------------------------------------------------------------

In [54]:

# Prompt Chaining 기법   이전 단계의 reasoning을 이용
# 앞의 판단 결과에 따라서 (승인/거절) 톤앤매너를 다르게 해서 이메일을 작성

# 승인 : 친절하고 빠르게 안내
# 거절 : 단호하게 거절하되 공감하는 톤

def draft_email(final_data):
    # 정보추출
    product = final_data['product']
    customer_issue = final_data['content']
    decison = final_data['decision_info']['decision']
    reason = final_data['decision_info']['reasoning']
    tone = '매우 정중하고 사과하는 태도' if decision == '승인' else '안타까워하며 공감하지만 원칙을 지키는 태도'

    prompt = f'''
        고객에게 보낼 이메일 작성 해줘

        상황 : 고객이 {product}에 대해 {customer_issue} 라고 문의
        결정 : {decison}
        이유 : {reason}
        작성 톤 : {tone}

        이메일 형식:
        제목 : 문의하신 {product}건에 대한 답변입니다.
        내용 : (인사 - 공감 - 결정 내용 통보 - 이유 설명 - 마무리)
    '''
    response = client.chat.completions.create(
        model = 'gpt-5-nano',
        messages=[{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content
for res in decision_results:
    email = draft_email(res)
    print(email)
    print('-'*100)

제목: 문의하신 무선 청소기건에 대한 답변입니다.

안녕하세요, 고객님.

먼저, 현재 상황으로 인해 불편을 겪으신 점 진심으로 안타깝게 생각합니다. 무선 청소기의 소음과 충전 이슈로 불편을 드려 대단히 죄송합니다.

고객님의 환불 요청에 대해 내부 정책에 따라 아래와 같이 안내드립니다.

결정 내용
- 환불은 규정상 처리해 드리기 어렵고, 대신 무상 AS를 통해 문제를 해결하는 것을 권장드립니다.

이유 설명
1) 단순 변심 환불(조항 1)은 박스가 온전해야 한다는 조건이 있습니다. 그러나 현재 박스가 없으므로 해당 조건에 부합하지 않습니다.
2) 제품 불량으로 인한 환불(조항 2)은 30일 이내에 가능하나, 구입 후 기간이 경과했습니다.
3) 불만족 보상(조항 3)은 환불은 불가하더라도 소정의 적립금 지급 여부를 검토할 수 있습니다. 다만 구체적 지급 조건은 운영 가이드에 따릅니다.
4) AS 규정(조항 4)은 구매 후 1년 이내 무상 AS가 가능하므로, 현 상황에서 가장 타당한 해결 경로는 AS를 통해 문제를 해결하는 것입니다.

AS 진행 안내
- AS 신청 시 필요 정보: 구매처/구매일, 주문번호(가능하면), 모델명 및 시리얼 번호, 문제 현상(소음 및 충전 이슈)의 상세 설명, 가능하면 문제 재현 영상이나 사진
- 접수 방법: 고객센터 또는 온라인 접수 채널을 통해 접수하실 수 있습니다. 접수 후 담당 팀이 수리 가능 여부와 일정 등을 안내해 드립니다.
- 처리 내용: 무상 AS 대상 부품 교체 또는 수리 등 상황에 맞는 조치를 진행합니다.

고객님의 편의를 위해 최대한 신속하고 원활하게 도와드리겠습니다. 추가로 필요하신 정보가 있거나 AS 접수를 원하시면 언제든지 연락 주시기 바랍니다.

감사합니다.

[회사명] 고객지원팀
----------------------------------------------------------------------------------------------------
제목: 문의하신 여름용 셔츠건에 대한 답변입니다